In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
ws.write_config(path='.azureml')
exp = Experiment(workspace=ws, name="quick-starts-ws-165838")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-167040
Azure region: southcentralus
Subscription id: d4ad7261-832d-46b2-b093-22156001df5b
Resource group: aml-quickstarts-167040


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
amlcompute_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, quniform
from azureml.core import ScriptRunConfig
from azureml.core import Environment
import os

# Specify parameter sampler
ps = RandomParameterSampling({'--C':uniform(0.1,1),
'--max_iter':quniform(100,1500,100)}

)

# Specify a Policy
policy = early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)


if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py

script_folder='./training'
os.makedirs(script_folder, exist_ok=True)
import shutil
shutil.copy('./train.py', script_folder)

env=Environment.get(workspace=ws, name="AzureML-Tutorial")

src= ScriptRunConfig(source_directory= script_folder,
                    script='train.py',
                    compute_target= aml_compute,
                     environment=env
                    )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config =HyperDriveConfig(run_config=src, 
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy", 
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                             max_total_runs=15,
                             max_concurrent_runs=3)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hdr=exp.submit(config=hyperdrive_config)

In [5]:
RunDetails(hdr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [9]:
import joblib
# Get your best run and save the model from that run.
best_run=hdr.get_best_run_by_primary_metric()
print(best_run) 
print(hdr.get_metrics())
### YOUR CODE HERE ###

Run(Experiment: quick-starts-ws-165838,
Id: HD_09168dbb-e21e-4a8b-af28-025e5c7f9594_2,
Type: azureml.scriptrun,
Status: Completed)
{'HD_09168dbb-e21e-4a8b-af28-025e5c7f9594_5': {'Regularization Strength:': 0.5664322534500552, 'Max iterations:': 1400, 'Accuracy': 0.9083459787556905}, 'HD_09168dbb-e21e-4a8b-af28-025e5c7f9594_4': {'Regularization Strength:': 0.2876607380926356, 'Max iterations:': 1200, 'Accuracy': 0.9081942336874052}, 'HD_09168dbb-e21e-4a8b-af28-025e5c7f9594_3': {'Regularization Strength:': 0.6241975799060206, 'Max iterations:': 500, 'Accuracy': 0.908649468892261}, 'HD_09168dbb-e21e-4a8b-af28-025e5c7f9594_2': {'Regularization Strength:': 0.3955366746766409, 'Max iterations:': 900, 'Accuracy': 0.9088012139605463}, 'HD_09168dbb-e21e-4a8b-af28-025e5c7f9594_1': {'Regularization Strength:': 0.4815441364727233, 'Max iterations:': 400, 'Accuracy': 0.908649468892261}, 'HD_09168dbb-e21e-4a8b-af28-025e5c7f9594_0': {'Regularization Strength:': 0.35255467588020906, 'Max iterations:':

In [10]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds=TabularDatasetFactory.from_delimited_files(["https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"])

In [13]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=66)

In [14]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=X_train,
    label_column_name='y',
    n_cross_validations=5)

In [15]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run=exp.submit(automl_config, show_output=False)
automl_run.wait_for_completion()

2021-12-20:16:18:10,775 INFO     [modeling_bert.py:226] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2021-12-20:16:18:10,797 INFO     [modeling_xlnet.py:339] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2021-12-20:16:18:33,231 INFO     [utils.py:157] NumExpr defaulting to 4 threads.
2021-12-20:16:18:33,492 WARNING  [local_experiment_driver.py:148] Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


DataException: DataException:
	Message: Expected column(s) y not found in X.
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Expected column(s) y not found in X.",
        "target": "X",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "MissingColumnsInData"
            }
        }
    }
}

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###